In [8]:
import sys
import os
sys.path.insert(0,'/home/prashant/unilm/layoutlmv3/examples/')
from gat_utils import *
# from ..examples.gat_utils import *

from PIL import Image, ImageDraw, ImageFont
import pickle
import numpy as np
import pandas as pd
import json
from collections import defaultdict
from datasets import load_metric 
metric = load_metric("seqeval")

In [5]:
train_dataset = pickle.load(open('/home/prashant/unilm/layoutlmv3/debug/funsd_train_processed_dataset.pkl','rb'))
test_dataset = pickle.load(open('/home/prashant/unilm/layoutlmv3/debug/funsd_test_processed_dataset.pkl','rb'))
# test_dataset_pre = pickle.load(open('/data1/prashant/DocIE/debug_v2/test_dataset_pre.pkl','rb'))

# id_to_image_test_json_ = json.load(open('/home/prashant/unilm/layoutlmv3/data/id_to_image_test.json'))
# id_to_image_test_json = {int(k):v for k,v in id_to_image_test_json_.items()}
image_to_id_test_json = json.load(open('/home/prashant/unilm/layoutlmv3/data/image_to_id_test.json'))
id_to_image_test_json = {j:i for i, j in image_to_id_test_json.items()}

# id_to_image_eval_json_ = json.load(open('/home/prashant/unilm/layoutlmv3/data/id_to_image_eval_funsd.json'))
# id_to_image_eval_json = {int(k):v for k,v in id_to_image_eval_json_.items()}
# image_to_id_eval_json = {j:i for i, j in id_to_image_eval_json.items()}

image_to_id_train_json = json.load(open('/home/prashant/unilm/layoutlmv3/data/image_to_id_train.json'))
# image_to_id_train_json = {int(k):v for k,v in id_to_image_train_json_.items()}
id_to_image_train_json = {int(j):i for i, j in image_to_id_train_json.items()}

few_shot_info = json.loads(open('/home/prashant/unilm/layoutlmv3/data/funsd_few_shot_info.json','r').read())
# image_to_id_dict = json.loads(open('/data1/prashant/DocIE/data/image_to_id.json','r').read())

# datasets = pickle.load(open('/data1/prashant/DocIE/data/datasets.pkl', 'rb'))

In [7]:
width_height_train, width_height_test = get_widths_heights(id_to_image_train_json, id_to_image_test_json)
width_height_list = [width_height_test[i] for i in test_dataset['overflow_to_sample_mapping']]

In [9]:
def get_filewise_metrics_csv(test_predictions, true_labels):
    filewise_metrics_csv = pd.DataFrame()
    row = defaultdict(float)
    tag_spec = ['precision','recall','f1','number']
    i = 0
    for preds, labels in zip(test_predictions, true_labels):
        row['id'] = i
        result = metric.compute(predictions=[preds], references=[labels])
        for k in ['ANSWER','QUESTION','HEADER']:
            for x in tag_spec:
                    try:
                        row[f"{k}_{x}"] = result[k][x]
                    except:
                        row[f"{k}_{x}"] = 0
        row['overall_precision'] = result['overall_precision']
        row['overall_recall'] = result['overall_precision']
        row['overall_f1'] = result['overall_f1']
        row['overall_accuracy'] = result['overall_accuracy']
        filewise_metrics_csv = filewise_metrics_csv.append(row, ignore_index=True)
        i+=1
    return filewise_metrics_csv

In [12]:
true_labels = np.genfromtxt('/home/prashant/unilm/layoutlmv3/data/funsd_true_labels.txt', dtype='str',delimiter=',,,')
true_labels_1 = [eval(x) for x in true_labels]

In [19]:
sizes = [1,2,3,4,5,6,7,8,9,10]
seeds = [0,1,2,3,4,5]
true_labels = np.genfromtxt('/home/prashant/unilm/layoutlmv3/data/funsd_true_labels.txt', dtype='str',delimiter=',,,')
true_labels_1 = [eval(x) for x in true_labels]
gat_closest_counts, gat_angles_counts = [],[]
for sz in sizes:
    print("Sz ", sz)
    closest_counts, angles_counts = [],[]
    for sd in seeds:
        test_fs_preds = np.genfromtxt(f'/home/prashant/unilm/layoutlmv3/results/test-layoutlmv3-fs-{sz}-{sd}/test_predictions.txt',dtype='str',delimiter=',,,,')
        test_gat_closest_preds = np.genfromtxt(f'/home/prashant/unilm/layoutlmv3/results/test-layoutlmv3-gat-closest-{sz}-{sd}-h4-d4/test_predictions.txt',dtype='str',delimiter=',,,,')
        test_gat_angles_preds = np.genfromtxt(f'/home/prashant/unilm/layoutlmv3/results/test-layoutlmv3-gat-60-angles-v2-{sz}-{sd}-h4-d4/test_predictions.txt',dtype='str',delimiter=',,,,')
        test_fs_preds = [x.split() for x in test_fs_preds]
        test_gat_closest_preds = [x.split() for x in test_gat_closest_preds]
        test_gat_angles_preds = [x.split() for x in test_gat_angles_preds]
        filewise_baseline = get_filewise_metrics_csv(test_fs_preds, true_labels_1)
        filewise_gat_closest = get_filewise_metrics_csv(test_gat_closest_preds, true_labels_1)
        filewise_gat_angles = get_filewise_metrics_csv(test_gat_angles_preds, true_labels_1)
        closest_counts.append(sum(filewise_baseline['overall_f1'] <= filewise_gat_closest['overall_f1']))
        angles_counts.append(sum(filewise_baseline['overall_f1'] <= filewise_gat_angles['overall_f1']))
    print(closest_counts)
    print(angles_counts)
    gat_closest_counts.append(np.mean(closest_counts))
    gat_angles_counts.append(np.mean(angles_counts))


Sz  1


/home/prashant/anaconda3/envs/layoutlmv3/lib/python3.7/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/prashant/anaconda3/envs/layoutlmv3/lib/python3.7/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/prashant/anaconda3/envs/layoutlmv3/lib/python3.7/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/prashant/anaconda3/envs/layoutlmv3/lib/p

[48, 30, 14, 31, 31, 35]
[37, 33, 17, 24, 24, 28]
Sz  2
[33, 40, 37, 34, 38, 33]
[31, 36, 35, 28, 36, 23]
Sz  3
[37, 39, 40, 31, 39, 32]
[37, 38, 39, 18, 34, 29]
Sz  4
[38, 37, 38, 26, 41, 26]
[36, 34, 33, 19, 40, 20]
Sz  5
[39, 34, 35, 26, 39, 27]
[37, 41, 35, 26, 33, 32]
Sz  6
[29, 36, 46, 26, 38, 23]
[26, 35, 41, 27, 41, 23]
Sz  7
[32, 37, 34, 41, 33, 30]
[31, 43, 33, 38, 34, 28]
Sz  8
[35, 30, 32, 39, 37, 40]
[31, 34, 34, 35, 33, 44]
Sz  9
[27, 36, 37, 38, 26, 38]
[26, 41, 37, 26, 32, 35]
Sz  10
[27, 37, 29, 35, 40, 31]
[29, 37, 35, 36, 42, 30]


In [24]:
print("FUNSD - closest")
avgs = []
for i, avg in enumerate(gat_closest_counts):
    print(f'Sz : {i+1} {avg} {avg/52}')
    avgs.append(avg/52)
print("AVG: ",np.mean(avgs[1:]))

avgs = []
print("FUNSD - angles")
for i, avg in enumerate(gat_angles_counts):
    print(f'Sz : {i+1} {avg} {avg/52}')
    avgs.append(avg/52)
print("AVG: ",np.mean(avgs[1:]))
    

FUNSD - closest
Sz : 1 31.5 0.6057692307692307
Sz : 2 35.833333333333336 0.6891025641025641
Sz : 3 36.333333333333336 0.6987179487179488
Sz : 4 34.333333333333336 0.6602564102564104
Sz : 5 33.333333333333336 0.6410256410256411
Sz : 6 33.0 0.6346153846153846
Sz : 7 34.5 0.6634615384615384
Sz : 8 35.5 0.6826923076923077
Sz : 9 33.666666666666664 0.6474358974358974
Sz : 10 33.166666666666664 0.6378205128205128
AVG:  0.6616809116809116
FUNSD - angles
Sz : 1 27.166666666666668 0.5224358974358975
Sz : 2 31.5 0.6057692307692307
Sz : 3 32.5 0.625
Sz : 4 30.333333333333332 0.5833333333333333
Sz : 5 34.0 0.6538461538461539
Sz : 6 32.166666666666664 0.6185897435897435
Sz : 7 34.5 0.6634615384615384
Sz : 8 35.166666666666664 0.6762820512820512
Sz : 9 32.833333333333336 0.6314102564102565
Sz : 10 34.833333333333336 0.6698717948717949
AVG:  0.6363960113960113
